## Importación de librerías

In [1]:
# Librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
import gzip

# Importar módulos propios del motor de red neuronal
from src.network import NeuralNetwork
from src.dense import Dense
from src.activations import ReLU, Softmax
from src.losses import CrossEntropyLoss
from src.optimizer import Adam
from src.trainer import Trainer

# Explicación: Esta celda importa todas las librerías necesarias para
# manejar datos, gráficos y la implementación de la red neuronal.


ModuleNotFoundError: No module named 'numpy'

## Funciones para cargar MNIST

In [ ]:
def load_mnist():
    """Carga las imágenes y etiquetas de MNIST desde archivos gzip"""
    def load_images(path):
        with gzip.open(path, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
            return data.reshape(-1, 28*28) / 255.0

    def load_labels(path):
        with gzip.open(path, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=8)
            return data

    X_train = load_images('mnist/train-images-idx3-ubyte.gz')
    y_train = load_labels('mnist/train-labels-idx1-ubyte.gz')
    X_test = load_images('mnist/t10k-images-idx3-ubyte.gz')
    y_test = load_labels('mnist/t10k-labels-idx1-ubyte.gz')
    return X_train, y_train, X_test, y_test

# Cargar datos
X_train, y_train, X_test, y_test = load_mnist()

# Explicación: Esta celda carga los datos de MNIST, normaliza las imágenes
# y devuelve los conjuntos de entrenamiento y prueba.


## Construcción de la red


In [ ]:
# Definir arquitectura de la red
network = NeuralNetwork([
    Dense(784, 128, initialization="he"),
    ReLU(),
    Dense(128, 10, initialization="xavier"),
    Softmax()
])

# Definir función de pérdida y optimizador
loss_fn = CrossEntropyLoss()
optimizer = Adam(lr=0.001)
trainer = Trainer(network, optimizer, loss_fn)

print("Red neuronal inicializada correctamente.")

# Explicación: Se define una red con una capa oculta de 128 neuronas
# y capa de salida de 10 neuronas, usando ReLU y Softmax.
# Se define la pérdida CrossEntropy y optimizador Adam.


## Entrenamiento






In [ ]:
# Parámetros de entrenamiento
num_epochs = 30
batch_size = 64

# Entrenar la red
history = trainer.train(
    X_train, y_train,
    X_test, y_test,
    epochs=num_epochs,
    batch_size=batch_size
)

# Explicación: Entrenamos la red 30 épocas usando batch de tamaño 64.
# Se guarda la historia de pérdida y exactitud para entrenamiento y validación.


## Graficar resultados


In [ ]:
plt.figure(figsize=(12,5))

# Pérdida
plt.subplot(1,2,1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()
plt.savefig('loss_plot.png')  # Guardar gráfico de pérdida

# Exactitud
plt.subplot(1,2,2)
plt.plot(history['train_acc'], label='Train Accuracy')
plt.plot(history['val_acc'], label='Validation Accuracy')
plt.title('Exactitud durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('accuracy_plot.png')  # Guardar gráfico de exactitud

plt.tight_layout()
plt.show()

# Explicación: Se muestran y guardan gráficos de pérdida y exactitud
# para entrenamiento y validación.


## Resultados finales


In [ ]:
print(f"Última pérdida de entrenamiento: {history['train_loss'][-1]:.4f}")
print(f"Última pérdida de validación: {history['val_loss'][-1]:.4f}")
print(f"Última exactitud de entrenamiento: {history['train_acc'][-1]*100:.2f}%")
print(f"Última exactitud de validación: {history['val_acc'][-1]*100:.2f}%")

# Explicación: Se muestran las métricas finales después de 30 épocas,
# útiles para resumir los resultados en la memoria.
